In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# create data VGG
## import libraries

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras.layers import AveragePooling2D, Input
from keras.utils import np_utils, normalize
from keras.engine import InputLayer
from keras import backend as K

import os
import scipy.io as sio
import h5py
import glob
import time

import sklearn
from sklearn.preprocessing import normalize
import random

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.linear_model import SGDClassifier


import matplotlib.pyplot as plt
import numpy as np
import time
import math

Using TensorFlow backend.


## modifiable variables

In [2]:
#dataset's path 
doc = 'cancer_cells'
# test size for microscope dataset
test_size = 0.3

dir_path = 'drive/My Drive/medical_image_recognition/datasets/'
save_path = 'drive/My Drive/medical_image_recognition/notebooks_Chen/'

vgg_fold='datanp_vgg/'
kmeans_fold="kmeans_PCA/"
vlad_fold="vlad_fold/"

scales=[0,1]
layers=['fc1']
# scales=[2]
# layers=['fc2']
pooling = 'mean'

# if save vectors in differents files
vgg_multi_files = True
RAM_SIZE = 4 * 1024 * 1024 * 1024 #GPU RAM Byte

flag_VLAD = False

#Kmeans
k_descriptors=30

print('Done')

Done


## functions

In [3]:
def read_dir_cancer(dir_path):
  files = sorted(os.listdir(dir_path))
  files = [dir_path+'/'+file for file in files]
  return files

def read_dir(dir_path) :
  """return the differents categories, the images in these categories, and the number of images per category"""
  listDir = sorted(os.listdir(dir_path))
  sizes = []
  listFiles = []
  for d in listDir :
    files = sorted(os.listdir(dir_path+'/'+d))
    sizes += [len(files)]
    listFiles += [files]
  return listDir, listFiles, sum(sizes)

def create_zero_array(layer, n, nb_images_left, nbImagesPerFile):
  """return a numpy array with zeros and with the needed scales"""
  nb = min(nb_images_left, nbImagesPerFile)
  if layer == 'block5_pool' :
    X = np.zeros((nb, 7 + 10*n, 7 + 10*n, 512))
  else :
    if n == 0 :
      X = np.zeros((nb, 4096))
    else :
      X = np.zeros((nb, 1 + 10*n, 1 + 10*n, 4096))
  return X


def complete_array_cancer(files, X_new, Y, n, n_image) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  nb = 0
  cpt = 0
  t0 = time.time()
  print(len(files))
  nb += len(files)
  Y_new = []
  cpt_X = 0
  for f in files :
    if cpt < n_image:
      cpt += 1
      continue
    if cpt_X == X_new.shape[0]:
        break
    t1 = time.time()
    img = image.load_img(f, target_size=(224 + 320*n, 224 + 320*n))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = network2.predict(x)
    X_new[cpt_X,:] = features[0,:]
    Y_new.append(Y[cpt])
    cpt += 1
    cpt_X += 1
    t2 = time.time()
    print('complete array{}/{}, time: {}s'.format(cpt,nb, t2-t1))
  print('total', t2 - t0)
  return X_new, Y_new


def complete_array(dir_path, listDir, listFiles, X, n, n_image) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  X_new = X
  Y = []
  nb = 0
  t0 = time.time()
  cpt_X = 0
  for i in range (len(listDir)):
    d = listDir[i]
    files = listFiles[i]
    print(d, len(files))
    cpt = nb
    nb += len(files)
    if nb < n_image:
      continue
    for f in files:
      if cpt < n_image:
        cpt += 1
        continue
      if cpt_X == X.shape[0]:
        break
      t1 = time.time()
      img = image.load_img(dir_path+'/'+d+'/'+f, target_size=(224 + 320*n, 224 + 320*n))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      features = network2.predict(x)
      X_new[cpt_X,:] = features[0,:]
      Y.append(d)
      cpt += 1
      cpt_X += 1
      t2 = time.time()
      print('{}/{}: {}'.format(cpt, nb, t2-t1))
  t2 = time.time()
  print('total time: {}'.format(t2 - t0))
  return X_new, Y

def to_fully_conv(model):
  """transforms the Convolutional Neural Network model into a Fully Convolutional Network"""

  new_model = Sequential()
  input_layer = InputLayer(input_shape=(None, None, 3), name="input_new")
  new_model.add(input_layer)

  for layer in model.layers:

    if "Flatten" in str(layer):
      flattened_ipt = True
      f_dim = layer.input_shape
      continue

    elif "Dense" in str(layer):

      input_shape = layer.input_shape
      output_dim =  layer.get_weights()[1].shape[0]
      W,b = layer.get_weights()

      if flattened_ipt:
          shape = (f_dim[1],f_dim[2],f_dim[3],output_dim)
          new_W = W.reshape(shape)
          aa = np.array([new_W,b])
          new_layer = Convolution2D(output_dim,(f_dim[1],f_dim[2]),
                                    activation=layer.activation,
                                    padding='valid', weights=[new_W,b])
          flattened_ipt = False

      else:
          shape = (1,1,input_shape[1],output_dim)
          new_W = W.reshape(shape)
          new_layer = Convolution2D(output_dim,(1,1),
                                    activation=layer.activation,
                                    padding='valid', weights=[new_W,b])
    else:
      new_layer = layer
    new_model.add(new_layer)
    
  return new_model

def pool(X, p):
  """return the numpy array X on which the pooling p was performed"""
  if(len(X.shape) == 2):
    return X
  if p == 'max' :  
    X_pool = np.max(X, axis = (1, 2)) 
  elif p == 'mean' :
    X_pool = np.mean(X, axis = (1, 2)) 
  else :
    X_pool = X
  return X_pool

def cal_nbSaveFiles(layer, n, size_train, size_test):
  if layer == 'block5_pool':
    size_image = 8*(7+10*n)*(7+10*n)*512
  else:
    size_image = 8*(1+10*n)*(1+10*n)*4096
    
  nbSaveFiles_train = math.ceil(size_image * size_train / RAM_SIZE)
  nbSaveFiles_test = math.ceil(size_image * size_test / RAM_SIZE)
  nbImagesPerFile = int(RAM_SIZE // size_image)
  return nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile


#path where the output array will be saved
def get_save_files_path(flag_VLAD, vgg_multi_files, save_path, doc, layer, n, pooling, nbSaveFiles_train, nbSaveFiles_test):
  
  path_save_train = []
  path_save_test = []
  path_save_Y_train = []
  path_save_Y_test = []
    
  if flag_VLAD == False:
    for i in range(nbSaveFiles_train):
      path_save_train.append(  save_path + 'X_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
      path_save_Y_train.append(save_path + 'y_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
    for i in range(nbSaveFiles_test):
      path_save_test.append(  save_path + 'X_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
      path_save_Y_test.append(save_path + 'y_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
    
  else:
    if vgg_multi_files:
      for i in range(nbSaveFiles_train):
        path_save_train.append(  save_path + 'X_train_' + layer + '(' + str(n) + ')'+ str(i)+'.npy')
        path_save_Y_train.append(save_path + 'y_train_' + layer + '(' + str(n) + ')'+str(i)+'.npy')
      for i in range(nbSaveFiles_test):
        path_save_test.append(  save_path + 'X_test_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
        path_save_Y_test.append(save_path + 'y_test_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
  return path_save_train, path_save_Y_train, path_save_test, path_save_Y_test

print('Done')

Done


In [0]:
dir_path_DataSet = dir_path + doc
save_path_Dataset = save_path + doc

#split train test examples
if doc == 'cancer_cells':

  dir_path_cancer = dir_path_DataSet  + '/cancer'
  dir_path_normal = dir_path_DataSet  + '/normal'

  #creating numpy array from images
  listFiles_cancer = read_dir_cancer(dir_path_cancer)
  listFiles_normal = read_dir_cancer(dir_path_normal)
  listFiles = listFiles_cancer + listFiles_normal

  Y_cancer = [1] * len(listFiles_cancer)
  Y_normal = [0] * len(listFiles_normal)
  Y = Y_cancer + Y_normal

  random.seed(42)
  test_index = random.sample(list(range(len(listFiles))), int(test_size*len(listFiles)))
  train_index = [x for x in list(range(len(listFiles))) if x not in test_index]
  
  listFiles_test = [listFiles[i] for i in test_index]
  listFiles_train = [listFiles[i] for i in train_index]
  Y_train = [Y[i] for i in train_index]
  Y_test = [Y[i] for i in test_index]
  size_train = len(listFiles_train)
  size_test = len(listFiles_test)
  
else:
  dir_path_train = dir_path_DataSet + '/train/'
  dir_path_test =  dir_path_DataSet + '/test/'

  listDir_train, listFiles_train, size_train = read_dir(dir_path_train)
  listDir_test, listFiles_test, size_test = read_dir(dir_path_test)


In [0]:

for layer in layers:
  for n in scales: 
    if vgg_multi_files:
      nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile = cal_nbSaveFiles(layer, n, size_train, size_test)
    else:
      nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile = 1, 1, max(size_train, size_test)
    
    network = VGG19(weights='imagenet')
    network3 = Model(inputs=network.input, outputs=network.get_layer(layer).output)
    if n >= 1 : 
      network2 = to_fully_conv(network3)
    else:
      network2 = network3
    print("layer = " + layer + " n = " + str(n) )
    
    path_save_train, path_save_Y_train, path_save_test, path_save_Y_test = get_save_files_path(flag_VLAD, vgg_multi_files, 
                                                                                               save_path_Dataset+'/'+vgg_fold, 
                                                                                               doc, layer, n, pooling, 
                                                                                               nbSaveFiles_train, nbSaveFiles_test)
    
      
    for i in range(nbSaveFiles_train):
      X_train = create_zero_array(layer, n, size_train-i*nbImagesPerFile, nbImagesPerFile)

      if doc == 'cancer_cells':
        X_train, Y_train = complete_array_cancer(listFiles_train, X_train, Y_train, n, i*nbImagesPerFile)
      else:
        X_train, Y_train = complete_array(dir_path_train, listDir_train, listFiles_train, X_train, n, i*nbImagesPerFile)
      if not flag_VLAD:
        X_train = pool(X_train, pooling)
      np.save(path_save_train[i], X_train)
      np.save(path_save_Y_train[i], Y_train)
      del X_train
      print('{}th train file saved'.format(i))
      
    for i in range(nbSaveFiles_test):
      X_test = create_zero_array(layer, n, size_test, nbImagesPerFile)
      if doc == 'cancer_cells':
        X_test, Y_test = complete_array_cancer(listFiles_test, X_test, Y_test, n, i*nbImagesPerFile)
      else:
        X_test, Y_test = complete_array(dir_path_test, listDir_test, listFiles_test, X_test, n, i*nbImagesPerFile)
      if not flag_VLAD:
        X_test = pool(X_test, pooling)
      np.save(path_save_test[i], X_test)
      np.save(path_save_Y_test[i], Y_test)
      del X_test
      print('{}th test file saved'.format(i))

    del network
    del network2
    del network3
print('Done')

Instructions for updating:
Colocations handled automatically by placer.
layer = fc2 n = 2
171
complete array1/171, time: 5.9658286571502686s
complete array2/171, time: 0.43146443367004395s
complete array3/171, time: 0.41449928283691406s
complete array4/171, time: 0.4179801940917969s
complete array5/171, time: 0.411452054977417s
complete array6/171, time: 0.4245717525482178s
complete array7/171, time: 0.43404674530029297s
complete array8/171, time: 0.42734599113464355s
complete array9/171, time: 0.41775035858154297s
complete array10/171, time: 0.4218556880950928s
complete array11/171, time: 0.4200780391693115s
complete array12/171, time: 0.4114952087402344s
complete array13/171, time: 0.43781614303588867s
complete array14/171, time: 0.42841649055480957s
complete array15/171, time: 0.42859935760498047s
complete array16/171, time: 0.438828706741333s
complete array17/171, time: 0.39345788955688477s
complete array18/171, time: 0.3983902931213379s
complete array19/171, time: 0.39044833183288

In [0]:
nbSaveFiles_test

1

# SVM without VLAD

In [8]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

if flag_VLAD == False:
  for layer in layers:
    for n in scales:
      print('layer: {}, scale: {}'.format(layer, n))
      nbSaveFiles_train, nbSaveFiles_test, nbImagesPerFile = cal_nbSaveFiles(layer, n, size_train, size_test)
      path_save_train, path_save_Y_train, path_save_test, path_save_Y_test = get_save_files_path(flag_VLAD, vgg_multi_files, 
                                                                                               save_path_Dataset+'/'+vgg_fold, 
                                                                                               doc, layer, n, pooling, 
                                                                                               nbSaveFiles_train, nbSaveFiles_test)
      
      Y_train = np.load(path_save_Y_train[0])
      Y_test = np.load(path_save_Y_test[0])
      X_train = np.load(path_save_train[0])
      X_test = np.load(path_save_test[0])
      for i in range(1, nbSaveFiles_train):
        X_train = np.concatenate((X_train, np.load(path_save_train[i])))
        Y_train = np.concatenate((Y_train, np.load(path_save_Y_train[i])))
      for i in range(1, nbSaveFiles_test):
        X_test = np.concatenate((X_test, np.load(path_save_test[i])))
        Y_test = np.concatenate((Y_test, np.load(path_save_Y_test[i])))

      # normalization
      X_train = normalize(X_train, norm = 'l2', axis = 1)
      X_test = normalize(X_test, norm = 'l2', axis = 1)

      """SUPPORT VECTOR MACHINE"""

      parameters = {'C':[0.00001,0.0001,0.001,0.01, 0.1, 1, 10,100]}
      svc = svm.SVC(kernel='linear')
      clf = GridSearchCV(svc, parameters, cv=3)
      clf.fit(X_train, Y_train)
      
      print('train set Accuracy: %.2f' % clf.score(X_train, Y_train))
      print('test set Accuracy: %.2f' % clf.score(X_test, Y_test))
      print('best params: {}'.format(clf.best_params_))
      print('Done')
      
print('Done')

      

layer: fc1, scale: 0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


train set Accuracy: 0.97
test set Accuracy: 0.68
best params: {'C': 10}
Done
layer: fc1, scale: 1


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


train set Accuracy: 1.00
test set Accuracy: 0.74
best params: {'C': 100}
Done
Done


In [0]:
X_train.shape

(120, 512)

In [0]:
X_test.shape

(120, 512)

# SVM with VLAD
## Kmeans

In [0]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

def save_file(path,obj):
  pickle_out = open(path,"wb")
  pickle.dump(obj, pickle_out)
  pickle_out.close()

In [0]:
for layer in layers:
  for n in scales:
    print("layer = " + layer + " n = " + str(n) )
    X=np.load(dir_path_DataSet + vgg_fold + 'X_train_'+layer + '(' + str(n) + ').npy')

    #this just for miniMIT bc we don't have many descriptorts 
    if doc=='miniMIT_Etus':
      if len(X.shape) == 4:
        k_descriptors = X.shape[1]*X.shape[2]
      else:
        k_descriptors = 1

    #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
    if len(X.shape) == 4:
      d=X.shape[1]*X.shape[2]
      X = np.reshape(X,(X.shape[0],d,X.shape[3]))
    else:
      d = 1
      X = np.reshape(X,(X.shape[0],d,X.shape[1]))

    #we take only some discriptors
    if doc!='miniMIT_Etus':
      #take random descriptors
      indexes = np.arange(d)
      np.random.shuffle(indexes)
      indexes=indexes[:k_descriptors]
      X=X[:,indexes,:]

    if(n==0):
      x_all=X[:]
    else:
      x_all=np.concatenate((x_all,X),axis=1)
    del X

  x_all=np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
  
  x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)

  x_all = StandardScaler(with_std=False).fit_transform(x_all)

  x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
  
  print("PCA fit transform ...")
  pca = PCA(n_components=128)
  x_train = pca.fit_transform(x_all)
  
  x_train = sklearn.preprocessing.normalize(x_train, norm = 'l2', axis = 1)

  #kmeans
  print("Kmean1 64 ...")
  kmeans1 = KMeans(n_clusters=64, random_state=0).fit(x_train)
  print("Kmean2 256 ...")
  kmeans2 = KMeans(n_clusters=256, random_state=0).fit(x_train)

  #save pca, kmeans  pickle
  save_file(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'PCA.pickle',pca)
  save_file(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'kmean1.pickle',kmeans1)
  save_file(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'kmean2.pickle',kmeans2)
        
  del x_all
print("END")
      

0 block5_pool
1 block5_pool
2 block5_pool
(15390, 512)


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


PCA fit ...
pca shape
(15390, 128)
Kmean1 64 ...
Kmean2 256 ...
end of  2 block5_pool
0 fc1
1 fc1
2 fc1
(10431, 4096)


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


PCA fit ...
pca shape
(10431, 128)
Kmean1 64 ...
Kmean2 256 ...
end of  2 fc1
END


## VLAD

In [0]:
import sklearn
from sklearn.cluster import KMeans
import numpy as np
from sklearn.svm import SVC
import pickle
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [0]:
def VLAD(X,visualDictionary):

  predictedLabels = visualDictionary.predict(X)
  centers = visualDictionary.cluster_centers_
  labels=visualDictionary.labels_
  k=visualDictionary.n_clusters

  m,d = X.shape
  V=np.zeros([k,d])
  #computing the differences

  # for all the clusters (visual words)
  for i in range(k):
    # if there is at least one descriptor in that cluster
    if np.sum(predictedLabels==i)>0:
      # add the diferences
      V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)

  V = V.flatten()
  # power normalization
  V = np.sign(V)*np.sqrt(np.abs(V))

  # L2 normalization
  V = V/np.sqrt(np.dot(V,V))
  return V
  
def get_vlad(layer,scales,kmean1,kmean2,pca,type_set="train"):
  for n in scales:
    print(n , layer)
    #laod_data
    if type_set=="train":
      x=np.load(dir_path_DataSet + vgg_fold +'X_train_'+layer + '(' + str(n) + ').npy')
    elif type_set=="test":
      x=np.load(dir_path_DataSet + vgg_fold +'X_test_'+layer + '(' + str(n) + ').npy')


    #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
    if len(x.shape) == 4:
      x = np.reshape(x,(x.shape[0],(x.shape[1]*x.shape[2]),x.shape[3]))
    else:
      x = np.reshape(x,(x.shape[0],1,x.shape[1]))

    #if x_all is empty : x_all take x else add data to x_all 
    if(n==0):
      x_all=x
    else:
      x_all=np.concatenate((x_all,x),axis=1)
                
    
    V_64=[]
    V_256=[]
    #for ech image
    for vec in x_all:
      #we take the first vec and we delete it from the data set: memory problem
      x_all=x_all[1:,...]
      #normalization l2
      vec = normalize(vec, norm = 'l2', axis = 1)
      #transformation with PCA
      vec=pca.transform(vec)
      #normalisation
      vec = normalize(vec, norm = 'l2', axis = 1)


      V_64.append(VLAD(vec,kmean1))
      V_256.append(VLAD(vec,kmean2))
    
    return V_64,V_256

In [0]:
for layer in layers:
  print('loading files')
  kmean1 = pickle.load(open(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'kmean1.pickle', 'rb'))
  kmean2 = pickle.load(open(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'kmean2.pickle', 'rb'))
  pca = pickle.load(open(dir_path_DataSet+kmeans_fold+'_'+layer+'_'+'PCA.pickle', 'rb'))

  print('VLAD for train set')
  v_train_64,v_train_256=get_vlad(layer,scales,kmean1,kmean2,pca,type_set="train")
  v_train_64 = np.asarray(v_train_64)
  v_train_256= np.asarray(v_train_256)

  np.save(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_train(64).npy',v_train_64)
  np.save(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_train(256).npy',v_train_256)
  del v_train_64,v_train_256

  print('VLAD for test set')
  v_test_64,v_test_256=get_vlad(layer,scales,kmean1,kmean2,pca,type_set="test")
  v_test_64 = np.asarray(v_test_64)
  v_test_256= np.asarray(v_test_256)

  np.save(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_test(64).npy',v_test_64)
  np.save(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_test(256).npy',v_test_256)
  del v_test_64,v_test_256

  print("done")

0 block5_pool
1 block5_pool
2 block5_pool
(171, 8192)
0 block5_pool
1 block5_pool
2 block5_pool
(73, 8192)
done
0 fc1
1 fc1
2 fc1
(171, 8192)
0 fc1
1 fc1
2 fc1
(73, 8192)
done
0 fc2
1 fc2
2 fc2
(171, 8192)
0 fc2
1 fc2
2 fc2
(73, 8192)
done


## SVM

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np

In [0]:
for layer in layers:
  print('prediction SVM [%s]' %(layer))
  
  v_train_64 = np.load(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_train(64).npy')
  v_train_256= np.load(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_train(256).npy')
  v_test_64  = np.load(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_test(64).npy')
  v_test_256 = np.load(dir_path_DataSet+vlad_fold+'_'+layer+'_'+'vlad_test(256).npy')

  y_train=np.load(dir_path_DataSet + vgg_fold + 'Y_train_'+layer + '(' + str(0) + ').npy')
  y_test=np.load( dir_path_DataSet + vgg_fold + 'Y_test_'+layer + '(' + str(0) + ').npy')
  
  parameters = {'C':[0.0001,0.001,0.01, 0.1, 1, 10,100]}
  
  print('SUPPORT VECTOR MACHINE 256')
  clf = SVC(kernel='linear')
  clf = GridSearchCV(clf, parameters, cv=3)
  clf.fit(v_train_256, y_train)
  print('Accuracy vlad 256: %.2f' % clf.score(v_test_256, y_test))
  
  print('SUPPORT VECTOR MACHINE 64')
  clf = SVC(kernel='linear')
  clf = GridSearchCV(clf, parameters, cv=3)
  clf.fit(v_train_64, y_train)
  print('Accuracy vlad 64: %.2f' % clf.score(v_test_64, y_test))        

(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.74
prediction SVM [block5_pool]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.77
(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.71
prediction SVM [fc1]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.77
(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.74
prediction SVM [fc2]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.79
